# Projet Kaggle - Titanic

https://www.kaggle.com/c/titanic/data

14/03/2018

Qui va vivre ?

In [1]:
#import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import svm, datasets


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [2]:
#importer les data du fichier train

df_train_raw = pd.read_csv('train.csv')
print('Shape : ')
print(df_train_raw.shape)
#df_train_raw.head(10)

Shape : 
(891, 12)


In [47]:
#importer les data du fichier test

df_test_raw = pd.read_csv('test.csv')
print('Shape : ')
print(df_test_raw.shape)
#df_test_raw.head(10)

Shape : 
(418, 11)


## Nettoyage

In [48]:
#Cleanage des data Train :

def clean_df_train(df_train_clean) :

    #df_train_clean['Sex'].replace('male', 0, inplace=True)
    #df_train_clean['Sex'].replace('female', 1, inplace=True)
    df_train_clean['Sex'].replace('', np.nan, inplace=True)
    df_train_clean.dropna(subset=['Sex'], inplace=True)

    df_train_clean['Age'].replace('', np.nan, inplace=True)
    df_train_clean.dropna(subset=['Age'], inplace=True)

    df_train_clean['Pclass'].replace(1, '1ere', inplace=True)
    df_train_clean['Pclass'].replace(2, '2eme', inplace=True)
    df_train_clean['Pclass'].replace(3, '3eme', inplace=True)
    df_train_clean['Pclass'].replace('', np.nan, inplace=True)
    df_train_clean.dropna(subset=['Pclass'], inplace=True)

    df_train_clean['SibSp'].replace('', np.nan, inplace=True)
    df_train_clean.dropna(subset=['SibSp'], inplace=True)

    df_train_clean['Parch'].replace('', np.nan, inplace=True)
    df_train_clean.dropna(subset=['Parch'], inplace=True)

    df_train_clean['Fare'].replace('', np.nan, inplace=True)
    df_train_clean.dropna(subset=['Fare'], inplace=True)

    #Remplacer le C, Q, S par 0, 1, 2
    #df_train_clean['Embarked'].replace('C', 0, inplace=True)
    #df_train_clean['Embarked'].replace('Q', 1, inplace=True)
    #df_train_clean['Embarked'].replace('S', 2, inplace=True)
    df_train_clean['Embarked'].replace('', np.nan, inplace=True)
    df_train_clean.dropna(subset=['Embarked'], inplace=True)
    
    return df_train_clean

In [49]:
df_train_clean = clean_df_train(df_train_raw)


In [50]:
# Travail sur la colonne Name

import re as re
import sys


def recup_titre(name):   
    
    titre = name
    
    try:    
        autre, titre = name.split(', ', maxsplit=1)
        titre, autre  = titre.split('.', maxsplit=1)
    except: 
        print(name)
    
    return titre


#Ajout des colonnes par Titre
def df_extract_titre(df_name):
    df_tempo = df_name
    df_name = df_name['Name'].map(lambda name : recup_titre(name) )    
    df_name = df_name.str.get_dummies()
    list_titre = df_name.columns
    df_name = pd.concat([df_tempo, df_name], axis=1, join='inner')
    return df_name, list_titre.tolist()

#df_testo, list_titre_temp = df_extract_titre(df_train_clean)
#list_titre_temp

In [51]:
# version 1
#list_col_One_Hot_Enco = ['Sex', 'Pclass', 'Embarked']
# version 2
list_col_One_Hot_Enco = [ 'Pclass', 'Embarked']

In [52]:
#Travail sur les colonnes OneHot Encoding


#Faire la liste des colonnes créées
def df_extract_one_hot(df_sex, col):
    df_tempo = df_sex
    df_sex = df_sex[col]
    df_sex = df_sex.str.get_dummies()
    nom_col = df_sex.columns
    df_sex = pd.concat([df_tempo, df_sex], axis=1, join='inner')
    return df_sex, nom_col

def df_extract_col_one_hot(df, listCol):
    list_colo = []
    for col in listCol :
        df, noms_col = df_extract_one_hot(df, col)
        list_colo = list_colo + noms_col.tolist()
        #list_colo = noms_col.tolist()
    return df, list_colo

#df_testy, listcolo = df_extract_one_hot(df_train_clean, 'Sex') 

df_testy, listcolo = df_extract_col_one_hot(df_train_clean, list_col_One_Hot_Enco)
listcolo

['1ere', '2eme', '3eme', 'C', 'Q', 'S']

In [53]:
# Clean df_train_raw + One Hot Encoding sur les autres col.

df_train_clean = clean_df_train(df_train_raw)
#df_train_clean = df_extract_titre(df_train_clean)
#df_train_clean = df_extract_col_one_hot(df_train_clean, list_col_One_Hot_Enco)
#df_train_clean = df_extract_col_one_hot(df_extract_titre(df_train_clean), list_col_One_Hot_Enco)

#df_train_clean.columns
df_train_clean.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3eme,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1ere,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3eme,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1ere,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3eme,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [55]:
#Split le fichier train pour garder une partie de test

#df_split_train, df_split_test = np.split(df_train_clean, [int(.8*len(df_train_clean))])
df_split_train, df_split_test = train_test_split(df_train_clean, test_size=0.2)

#df_train_clean = df_split_train


In [56]:
#Appliquer le nettoyage sur le train
df_train, liste_col_One_All = df_extract_col_one_hot(df_split_train, list_col_One_Hot_Enco)
df_train, liste_titre_All = df_extract_titre(df_train)

In [57]:
#Colonnes utilisées pour le calcul
#list_col_train = list(['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked'])
list_col_train = list(['Age','SibSp','Parch','Fare'])

#Ajoutons la liste des titres
list_col_train = list_col_train + liste_col_One_All + liste_titre_All

list_col_train

['Age',
 'SibSp',
 'Parch',
 'Fare',
 '1ere',
 '2eme',
 '3eme',
 'C',
 'Q',
 'S',
 'Capt',
 'Col',
 'Don',
 'Dr',
 'Jonkheer',
 'Lady',
 'Major',
 'Master',
 'Miss',
 'Mme',
 'Mr',
 'Mrs',
 'Ms',
 'Rev',
 'Sir']

## Training

In [58]:
df_train.head(4)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,Lady,Major,Master,Miss,Mme,Mr,Mrs,Ms,Rev,Sir
296,297,0,3eme,"Hanna, Mr. Mansour",male,23.5,0,0,2693,7.2292,...,0,0,0,0,0,1,0,0,0,0
635,636,1,2eme,"Davis, Miss. Mary",female,28.0,0,0,237668,13.0000,...,0,0,0,1,0,0,0,0,0,0
396,397,0,3eme,"Olsson, Miss. Elina",female,31.0,0,0,350407,7.8542,...,0,0,0,1,0,0,0,0,0,0
787,788,0,3eme,"Rice, Master. George Hugh",male,8.0,4,1,382652,29.1250,...,0,0,1,0,0,0,0,0,0,0


In [59]:
#Extract des colonnes pour le calcul
list_clean = df_train[list_col_train].values.tolist()
tab_clean = np.array(list_clean)

list_survived = df_train[['Survived']].values.tolist()
tab_survived = np.array(list_survived)

print(tab_clean.shape)
print(tab_survived.shape)

(569, 25)
(569, 1)


In [ ]:
#Logistical Reg
from sklearn.linear_model import LogisticRegression

logisticRegr_All = LogisticRegression()

logisticRegr_All.fitit(tab_clean, tab_survived.ravel())

In [61]:
df_train[list_col_train].to_csv('df_train - eb.csv')
df_train[['Survived']].to_csv('df_train_surv - eb.csv')

In [ ]:
list_tab_coef = logisticRegr_All.coef_
tab_coef = list_tab_coef[0]
list_coef = []
for i in tab_coef :
    list_coef.append(i)
    
list_coef

df_coef = pd.DataFrame(list_coef, columns=['Coef'], index = list_col_train)
df_coef

In [ ]:
df_coef.hist(column='Coef')

In [ ]:
#Score sur le même préset
logisticRegr_All.score(tab_clean, tab_survived.ravel())

In [ ]:
df_train['Pclass'].hist()

## Validation

In [ ]:
def df_extract_titre_test(df, list_titre):
    df_temp, list_temp = df_extract_titre(df)
    
    col_vide = [0] * df['Name'].count()  
    
    #Ajouter des colonnes vide selon les models
    for i in list_titre :
        
        try :             
            df_temp[[i]]            
        except : 
            df_temp[i] = col_vide
        
           
    return df_temp

#df_extract_titre_test(df_test, liste_titre_All)

In [ ]:
#Appliquer le nettoyage sur le split_test
df_test, liste_col_One_TEST = df_extract_col_one_hot(df_split_test, list_col_One_Hot_Enco)
df_test = df_extract_titre_test(df_test, liste_titre_All)

In [ ]:
#Score sur le split de train

list_split_test = df_test[list_col_train].values.tolist()
tab_split_test = np.array(list_split_test)

list_split_test_survived = df_test[['Survived']].values.tolist()
tab_split_test_survived = np.array(list_split_test_survived)


logisticRegr_All.score(tab_split_test, tab_split_test_survived.ravel())

## Test

In [ ]:
#Nettoyage des data Test CSV

def clean_df_test(df_test_clean, df_split_train) :

    #df_test_clean['Sex'].replace('male', 0, inplace=True)
    #df_test_clean['Sex'].replace('female', 1, inplace=True)
    #df_test_clean['Sex'].replace('', np.nan, inplace=True)
    #df_test_clean['Sex'].replace(np.nan, df_split_train['Sex'].median(), inplace=True)
    #df_test_clean.dropna(subset=['Sex'], inplace=True)

    
    #med = df_split_train['Age'].median()    
    #print(med)
    
    df_test_clean['Age'].replace('', np.nan, inplace=True)  
    df_test_clean['Age'].replace(np.nan, df_split_train['Age'].median(), inplace=True)
    #df_test_clean.dropna(subset=['Age'], inplace=True)

    df_test_clean['Pclass'].replace(1, '1ere', inplace=True)
    df_test_clean['Pclass'].replace(2, '2eme', inplace=True)
    df_test_clean['Pclass'].replace(3, '3eme', inplace=True)    
    df_test_clean['Pclass'].replace('', np.nan, inplace=True)
    #Au lieu de la médiane comme valeur, je mets '3eme' car selon l'histo du train, il y a plus de 3eme
    df_test_clean['Pclass'].replace(np.nan, '3eme', inplace=True)
    #df_test_clean['Pclass'].replace(np.nan, df_split_train['Pclass'].median(), inplace=True)
    #df_test_clean.dropna(subset=['Pclass'], inplace=True)

    df_test_clean['SibSp'].replace('', np.nan, inplace=True)
    df_test_clean['SibSp'].replace(np.nan, df_split_train['SibSp'].median(), inplace=True)
    #df_test_clean.dropna(subset=['SibSp'], inplace=True)

    df_test_clean['Parch'].replace('', np.nan, inplace=True)
    df_test_clean['Parch'].replace(np.nan, df_split_train['Parch'].median(), inplace=True)
    df_test_clean.dropna(subset=['Parch'], inplace=True)

    df_test_clean['Fare'].replace('', np.nan, inplace=True)
    df_test_clean['Fare'].replace(np.nan, df_split_train['Fare'].median(), inplace=True)
    #df_test_clean.dropna(subset=['Fare'], inplace=True)

    #Remplacer le C, Q, S par 0, 1, 2
    #df_test_clean['Embarked'].replace('C', 0, inplace=True)
    #df_test_clean['Embarked'].replace('Q', 1, inplace=True)
    #df_test_clean['Embarked'].replace('S', 2, inplace=True)
    #df_test_clean['Embarked'].replace('', np.nan, inplace=True)
    #df_test_clean['Embarked'].replace(np.nan, df_split_train['Embarked'].median(), inplace=True)
    #df_test_clean.dropna(subset=['Embarked'], inplace=True)
    
    return df_test_clean

In [ ]:
#Appliquer le nettoyage sur le split_test
df_test_CSV = clean_df_test(df_test_raw, df_train)
df_test_CSV, liste_col_One_TEST = df_extract_col_one_hot(df_test_CSV, list_col_One_Hot_Enco)
df_test_CSV = df_extract_titre_test(df_test_CSV, liste_titre_All)

df_test_CSV

In [ ]:
#Récupérer la colonne Passenger ID
list_test_CSV_passengerID = df_test_CSV[['PassengerId']].values.tolist()
tab_test_CSV_passengerID = np.array(list_test_CSV_passengerID)

tab_test_CSV_passengerID.size

In [ ]:
# Extraction des colonnes utilisées par le train
list_test= df_test_CSV[list_col_train].values.tolist()
tab_test = np.array(list_test)

#list_test

In [ ]:
#prédiction
tab_predict = logisticRegr_All.predict(tab_test)


#Compter les survivants
cond = tab_predict == 1

print(tab_predict[cond].sum())
print(tab_predict.shape)

In [ ]:
df_result = pd.DataFrame(tab_test_CSV_passengerID.tolist(), columns=['PassengerId'] )
df_result['Survived'] = tab_predict
df_result.head(5)

In [ ]:
#exporter vers file
df_result.to_csv('result v2.csv', index = False, header=True)

# Scores Kaggles
    0.76555 : ['Age','SibSp','Parch','Fare'] + Name + list_col_One_Hot_Enco = ['Sex', 'Pclass', 'Embarked'] 
    0.77033 : ['Age','SibSp','Parch','Fare'] + Name + list_col_One_Hot_Enco = ['Pclass', 'Embarked']